With the bandit selection models, we evaluate their performances on the test set

In [1]:
from dataset import *
from train import *

In [2]:
import os
import pandas as pd

import collections

import sys

import tarfile
import torch
import torch.optim as optim

from pathlib import Path
import glob

In [3]:
input_path = Path("derived_data/chtc_0730/")
output_path = input_path / "test_results"
output_path.mkdir(parents = True, exist_ok = True)

In [4]:
root = 'ds/images/remote_sensing/otherDatasets/sentinel_2/tif/'
download_ON = os.path.exists(root)
if not download_ON:
  os.system('wget http://madm.dfki.de/files/sentinel/EuroSATallBands.zip') #All bands
  !unzip EuroSATallBands.zip
  download_ON = True
    
geo_df = pd.read_csv("metadata.csv")

data = torchvision.datasets.DatasetFolder(root=root,loader = iloader, transform = None, extensions = 'tif')
labels = [v[1] for (i, v) in enumerate(data)]

In [5]:
test_results = []
for tar_path in glob.glob(str(input_path) + "/*tar.gz"):
    if "United_Kingdom" in tar_path:
        continue
    file = tarfile.open(tar_path)
    file.extractall(output_path)
    
    model_path = glob.glob(str(output_path) + "/*/*pt")[0]
    
    model = Load_model()
    model.load_state_dict(torch.load(model_path))
    
    [target_task, algorithm, target_size] = tar_path.split("/")[-1].split(".")[0].split("derived_data_")[-1].split("_")
    target_size = int(target_size)
    input_data = prepare_input_data(geo_df, target_task, labels = labels,
                               target_size = target_size)
    
    np.random.seed(0)
    torch.cuda.manual_seed(0)
    random.seed(0)

    # prepare data ---

    target_test_loader =  torch.utils.data.DataLoader(torch.utils.data.Subset(data, input_data["idx_test"]), 
                                                      batch_size = 16, shuffle = True, num_workers = 0)


    test_acc = validation(model, target_test_loader)
    test_results.append({
        "target_task": target_task,
        "target_size": target_size,
        "algorithm": algorithm,
        "test_acc": test_acc
    })
    
    os.system("rm -r " + str(output_path) + "/*")
    

Model Loaded
Model Loaded
Model Loaded
Model Loaded
Model Loaded
Model Loaded
Model Loaded
Model Loaded
Model Loaded
Model Loaded
Model Loaded
Model Loaded
Model Loaded
Model Loaded
Model Loaded
Model Loaded
Model Loaded
Model Loaded
Model Loaded
Model Loaded
Model Loaded
Model Loaded
Model Loaded
Model Loaded


In [6]:
pd.concat([pd.DataFrame(r, index = [0]) for r in test_results]).to_csv(input_path / "test_acc.csv")